<a href="https://colab.research.google.com/github/hashemi124/ANN_edu_deep/blob/master/mvision_cal101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#from keras.datasets import mnist
from keras import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam,SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import glob
from skimage import io
import os
import scipy.misc

from keras import regularizers
import csv
from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
datasets_path = '/content/drive/My Drive/Colab Notebooks/Caltech101/train' #Add the path to the unzipped folder

def load_images(path,n=0):
    X = []
    Y=[]
    i=-1
    labels = []
    for label in os.listdir(path):
        back_path = os.path.join(path,label)
        labels.append(label)
        i = i+1
        for filename in os.listdir(back_path):
            image_path = os.path.join(back_path,filename)
            img = image.load_img(image_path,target_size=(224,224))
            img = image.img_to_array(img)
            #Y.append(image)
            #image = imresize(image,[128,128,3])
            #image = imresize(imread(image_path), [128,128, 3])
            #image = image.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            #image = image/255
            #image = 1-image
            Y.append(i)
            X.append(img)
            #X.append(image.img_to_array(img))
            #X.append(image)
    return X,Y,labels

x_train,y,label_data = load_images(datasets_path)
X = np.array(x_train)
#print(len(x_train))
#print(x_train.shape)
Y = np.array(y)
print(X.shape)

(450, 224, 224, 3)


In [14]:
datasets_path = '/content/drive/My Drive/Colab Notebooks/Caltech101/test' #Add the path to the unzipped folder

def load_images(path,n=0):
    X = []
    Y=[]
    i=-1
    labels = []
    for label in os.listdir(path):
        back_path = os.path.join(path,label)
        labels.append(label)
        i = i+1
        for filename in os.listdir(back_path):
            image_path = os.path.join(back_path,filename)
            img = image.load_img(image_path,target_size=(224,224))
            img = image.img_to_array(img)
            #Y.append(image)
            #image = imresize(image,[128,128,3])
            #image = imresize(imread(image_path), [128,128, 3])
            #image = image.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            #image = image/255
            #image = 1-image
            Y.append(i)
            X.append(img)
            #X.append(image.img_to_array(img))
            #X.append(image)
    return X,Y,labels

x,y,label_data = load_images(datasets_path)
X_test= np.array(x)
#print(len(x_train))
#print(x_train.shape)
Y_test = np.array(y)
print(X_test.shape)

(2856, 224, 224, 3)


In [18]:
from keras.utils.np_utils import to_categorical   

In [27]:
y_train = to_categorical(Y, num_classes=15)
y_test = to_categorical(Y_test, num_classes=15)

In [25]:
X_train = np.reshape(X,(450,150528))
X_test = np.reshape(X_test,(2856,150528))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalization of pixel values (to [0-1] range)

X_train /= 255
X_test /= 255

In [28]:
print("Shape of training data:")
print(X_train.shape)
print(y_train.shape)
print("Shape of test data:")
print(X_test.shape)
print(y_test.shape)

Shape of training data:
(450, 150528)
(450, 15)
Shape of test data:
(2856, 150528)
(2856, 15)


In [29]:
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=sgd)